In [225]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import tqdm as tqdm
#from selenium.webdriver.firefox.options import Options
#from selenium.webdriver.firefox.service import Service

In [226]:
chrome_options = Options()
firefox_driver_path = 'C:/webdrivers/geckodriver.exe'
chrome_driver_path = 'C:/webdrivers/chromedriver.exe'
service = Service(chrome_driver_path)
browser = webdriver.Chrome(options=chrome_options, service=service) # For Chrome
#browser = webdriver.Firefox(options=chrome_options, service = service)

#Empty lists to collect the data from the tables
list_of_rows_odd = [] 
list_of_rows_even = []

urls = [] # List of all the page urls for scraping that we need from 2015 - 2021

In [227]:
# Generates our list of urls to parse into BS Url. 
def parse_url():  
    year = [*range(2015, 2022)]
    year.insert(2, 2016) # Inserting a repeat year as Max drove for both Torro Rosso And Red Bull in 2016
    year_check = 0
    for y in year:
        if y == 2015 or 2016 and year_check < 2 :
            constructor = 'toro-rosso'
            year_check += 1
        else:   
            constructor = 'red-bull'
        urls.append(f'https://www.statsf1.com/en/{constructor}/grand-prix-{y}.aspx')
    return urls


In [228]:
# Loads the page and gets the data from the table by the get_table_data_by_row() func call
def load_page(urls):
    parse_url()
    for url in urls: 
        browser.get(url)
        soup = BeautifulSoup(browser.page_source, 'html.parser')
        get_table_data_by_row(1, 2, list_of_rows_odd, soup)
        get_table_data_by_row(2, 2, list_of_rows_even, soup)
        
    return    

In [229]:
# Gets the table data by alternating rows to tackle the row span discrepency
def get_table_data_by_row(starting_row, row_increment, list_to_append, soup_):
    row_num = starting_row
    while True:
        element = soup_.select(f"tr:nth-of-type({row_num})")
        if len(element) >= 1:
            for i in range(0, len(element)):
                tr = element[i]
                tmp_list = []
                td = tr.select('tr > td')
                for y in range(0, len(td)):
                    tmp_list.append(td[y].text.strip())
                list_to_append.append(tmp_list)
            row_num += row_increment
        else:
            break

# <p align=center>Scraping The Data </p>


In [230]:
load_page(urls)

# <p align=center> Pre Processing Data </p>
## <p>Noticing that the data from the website is displayed rather annoyingly, in the form of span rows, I have decided to alternate the rows with the view of pasting together both list into a DataFrame. To acheieve this, I have to pre process the data to have it in the correct sizing, format to import into the DataFrame.  The odd rows in particular, the way the data was displayed resulted in some entries being not of the same length, thus being out of sync with most of the data pulled from this subsection of rows.  The best way I came about to solving this issue, was to isolate the rows that are not needed, in particular the empty lists, and those that took the header of the table, and remove them.  Then insert a generic entry into the lists that remain, to align the data with the rest.  I will go through and clean this data once It is active in the DataFrame</p>

In [231]:
odd_df = pd.DataFrame(list_of_rows_odd, columns=["id", "track", "team", "driver_num", "driver", "model", "engine", "tyre", "grid_pos", "fin_pos", "notes"]) 
print(odd_df.shape)

odd_df = odd_df.iloc[odd_df.index[46:68]].shift(periods=2, axis="columns")


(172, 11)


In [232]:
# Sanity check for the amount of lists with empty strings. 
len(list_of_rows_odd)
count = 0
for ls in list_of_rows_odd: 
    if ls[0] == '':
        count += 1
print(count)

8


In [233]:
tmp_list_of_rows_odd = list_of_rows_odd.copy()  # Making a copy of the list to have a backup incase something goes wrong.

In [241]:
# Removing lists of empty strings that we have in the list, and the lists of table headers from the webpage, and also insterting generic data into the indices to make it uniform
for ls in tmp_list_of_rows_odd:
    if ls[0] == '':
        tmp_list_of_rows_odd.remove(ls)
    if ls[0] == 'n':
        tmp_list_of_rows_odd.remove(ls)
    if len(ls) < 10:
        ls.insert(0,'id')
        ls.insert(1,'track')
        print(ls)
    else:
        pass
#print(len(tmp_list_of_rows_odd))

['id', 'track', 'Red Bull Racing', '26', 'KVYAT Daniil', 'RB12', 'TAG Heuer', 'Pirelli', '15', '7', '']
['id', 'track', 'Red Bull Racing', '26', 'KVYAT Daniil', 'RB12', 'TAG Heuer', 'Pirelli', '6', '3', '']
['id', 'track', 'Red Bull Racing', '26', 'KVYAT Daniil', 'RB12', 'TAG Heuer', 'Pirelli', '8', '15', '']
['id', 'track', 'Red Bull Racing', '33', 'VERSTAPPEN Max', 'RB12', 'TAG Heuer', 'Pirelli', '4', '1', '']
['id', 'track', 'Red Bull Racing', '33', 'VERSTAPPEN Max', 'RB12', 'TAG Heuer', 'Pirelli', '21', 'ab', 'Accident']
['id', 'track', 'Red Bull Racing', '33', 'VERSTAPPEN Max', 'RB12', 'TAG Heuer', 'Pirelli', '5', '4', '']
['id', 'track', 'Red Bull Racing', '33', 'VERSTAPPEN Max', 'RB12', 'TAG Heuer', 'Pirelli', '9', '8', '']
['id', 'track', 'Red Bull Racing', '33', 'VERSTAPPEN Max', 'RB12', 'TAG Heuer', 'Pirelli', '8', '2', '']
['id', 'track', 'Red Bull Racing', '33', 'VERSTAPPEN Max', 'RB12', 'TAG Heuer', 'Pirelli', '3', '2', '']
['id', 'track', 'Red Bull Racing', '33', 'VERSTAP

In [242]:
tmp_list_of_rows_odd

[['167',
  'Australia',
  'Scuderia Toro Rosso',
  '33',
  'VERSTAPPEN Max',
  'STR10',
  'Renault',
  'Pirelli',
  '11',
  'ab',
  'Engine'],
 ['168',
  'Malaysia',
  'Scuderia Toro Rosso',
  '33',
  'VERSTAPPEN Max',
  'STR10',
  'Renault',
  'Pirelli',
  '6',
  '7',
  ''],
 ['169',
  'China',
  'Scuderia Toro Rosso',
  '33',
  'VERSTAPPEN Max',
  'STR10',
  'Renault',
  'Pirelli',
  '13',
  '17',
  'Engine'],
 ['170',
  'Bahrain',
  'Scuderia Toro Rosso',
  '33',
  'VERSTAPPEN Max',
  'STR10',
  'Renault',
  'Pirelli',
  '15',
  'ab',
  'Electrics'],
 ['171',
  'Spain',
  'Scuderia Toro Rosso',
  '33',
  'VERSTAPPEN Max',
  'STR10',
  'Renault',
  'Pirelli',
  '6',
  '11',
  ''],
 ['172',
  'Monaco',
  'Scuderia Toro Rosso',
  '33',
  'VERSTAPPEN Max',
  'STR10',
  'Renault',
  'Pirelli',
  '9',
  'ab',
  'Collision'],
 ['173',
  'Canada',
  'Scuderia Toro Rosso',
  '33',
  'VERSTAPPEN Max',
  'STR10',
  'Renault',
  'Pirelli',
  '19',
  '15',
  ''],
 ['174',
  'Austria',
  'Scuderi

# <p align=center>Creating the first DataFrame</p>
## Now that we have pre processed our odd data, by adding the requried indices to make it conform to the main list we can create our first DataFrame.

In [243]:
odd_df = pd.DataFrame(tmp_list_of_rows_odd, columns=["id", "track", "team", "driver_num", "driver", "model", "engine", "tyre", "grid_pos", "fin_pos", "notes"]) 
odd_df.head(50)

,id,track,team,driver_num,driver,model,engine,tyre,grid_pos,fin_pos,notes
0,167,Australia,Scuderia Toro Rosso,33,VERSTAPPEN Max,STR10,Renault,Pirelli,11,ab,Engine
1,168,Malaysia,Scuderia Toro Rosso,33,VERSTAPPEN Max,STR10,Renault,Pirelli,6,7,
2,169,China,Scuderia Toro Rosso,33,VERSTAPPEN Max,STR10,Renault,Pirelli,13,17,Engine
3,170,Bahrain,Scuderia Toro Rosso,33,VERSTAPPEN Max,STR10,Renault,Pirelli,15,ab,Electrics
4,171,Spain,Scuderia Toro Rosso,33,VERSTAPPEN Max,STR10,Renault,Pirelli,6,11,
5,172,Monaco,Scuderia Toro Rosso,33,VERSTAPPEN Max,STR10,Renault,Pirelli,9,ab,Collision
6,173,Canada,Scuderia Toro Rosso,33,VERSTAPPEN Max,STR10,Renault,Pirelli,19,15,
7,174,Austria,Scuderia Toro Rosso,33,VERSTAPPEN Max,STR10,Renault,Pirelli,7,8,
8,175,Britain,Scuderia Toro Rosso,33,VERSTAPPEN Max,STR10,Renault,Pirelli,13,ab,Accident
9,176,Hungary,Scuderia Toro Rosso,33,VERSTAPPEN Max,STR10,Renault,Pirelli,9,4,
